## Test Framework Demo
1. Define map (`CENTRE`, `RES`), number of targets (`NUM_CASUALTY`), and number of search iterations (`STEPS`)
2. Instantiate TestFramework object
3. Register pathfinder
4. Generate casualty
5. Run test
6. Evaluate
7. Visualize

In [1]:
import h3
import folium
import numpy as np
import os
import random

from pathfinder.algo import (
    OutwardSpiralPathFinder, 
    BayesianHexSearch
)
from utils.hex import (
    hex_to_binary, 
    octal_list_to_binary, 
    binary_to_octal_list, 
    binary_to_hex, 
    hex_to_array_index,
    array_index_to_hex,
    distance_between_2_hexas,
)
from utils.viz import (
    gradient_color,
    add_hex_to_map,
    visualise_hex_dict_to_map,
    create_gif
)
from experiment.test_framework import TestFramework

CENTRE = (1.340640225800291, 103.962424829341)
HOTSPOT1 = (1.340647264373615, 103.9623395934375)
HOTSPOT2 = (1.340590906845766, 103.96242415287534)
RES = 15
NUM_CASUALTY = 5
STEPS = 200
random.seed(0)

## Spiral

In [2]:
test_spiral = TestFramework(name="Spiral", centre=CENTRE, res=RES)
test_spiral.register_pathfinder(OutwardSpiralPathFinder)

test_spiral.add_hotspot(HOTSPOT1)
test_spiral.add_hotspot(HOTSPOT2)

test_spiral.generate_casualty(NUM_CASUALTY)
spiral_output = test_spiral.run(STEPS)
test_spiral.evaluate()



Spiral's Path Coverage: 27.74%
Spiral's Guaranteed Capture: NO, captured 3/5
Spiral's False Negative: 1
Spiral's Minimum Time Capture: NA


In [3]:
# visualise_hex_dict_to_map(test_spiral.probability_map, m, test_spiral.casualty_locations)
m = folium.Map(location=[CENTRE[0],CENTRE[1]], zoom_start=23, max_zoom=25)
hex_hotspot1 = h3.geo_to_h3(HOTSPOT1[0],HOTSPOT1[1], resolution = RES)
hex_hotspot2 = h3.geo_to_h3(HOTSPOT2[0],HOTSPOT2[1], resolution = RES)

visualise_hex_dict_to_map(test_spiral.probability_map, m, {hex_hotspot1:1, hex_hotspot2:1})

display(m)

In [4]:
add_hex_to_map(spiral_output, m)
display(m)

## Bayes

In [5]:
f = 0.99
test_bayes = TestFramework(name="Bayes", centre=CENTRE, res=RES, f=f)
test_bayes.register_pathfinder(BayesianHexSearch)

test_bayes.add_hotspot(HOTSPOT1)
test_bayes.add_hotspot(HOTSPOT2)

test_bayes.generate_casualty(NUM_CASUALTY)
bayes_output = test_bayes.run(STEPS, update_map=True)
test_bayes.evaluate()

m = folium.Map(location=[CENTRE[0],CENTRE[1]], zoom_start=23, max_zoom=25)

visualise_hex_dict_to_map(test_bayes.probability_map, m, test_bayes.casualty_locations)
add_hex_to_map(bayes_output, m)

display(m)

Bayes's Path Coverage: 26.77%
Bayes's Guaranteed Capture: YES, captured 5/5
Bayes's Minimum Time Capture: 0.028153 seconds


## Create GIF

In [6]:
# if STEPS >= 200:
#     print("This gna take forever")
# output_path = os.path.join(os.getcwd(), "Spiral_path.gif")
# create_gif(output_path, test_spiral.probability_map, spiral_output, test_spiral.casualty_locations, test_spiral.casualty_detected, dpi=50)

# output_path = os.path.join(os.getcwd(), "Bayes_path.gif")
# create_gif(output_path, test_bayes.probability_map, bayes_output, test_bayes.casualty_locations, test_bayes.casualty_detected, dpi=50)